In [2]:
import re

def parse_input(input_lines):
    """
    Parses the input lines to extract clay vein positions.

    Args:
        input_lines (list of str): Each string represents a clay vein definition.

    Returns:
        tuple: A tuple containing:
            - clay (set of tuples): Set of (x, y) positions occupied by clay.
            - min_x (int): Minimum x-coordinate in the clay veins.
            - max_x (int): Maximum x-coordinate in the clay veins.
            - min_y (int): Minimum y-coordinate in the clay veins.
            - max_y (int): Maximum y-coordinate in the clay veins.
    """
    clay = set()
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line formats:
        # "x=495, y=2..7"
        # "y=7, x=495..501"
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue  # Skip lines that don't match the expected format
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_x, max_x, min_y, max_y

def print_x_labels(min_x, max_x):
    """
    Prints the x-axis labels above the grid for reference.

    Args:
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
    """
    # Create the three label lines: hundreds, tens, units
    hundreds = ""
    tens = ""
    units = ""
    for x in range(min_x, max_x + 1):
        hundreds += str(x // 100)
    for x in range(min_x, max_x + 1):
        tens += str((x % 100) // 10)
    for x in range(min_x, max_x + 1):
        units += str(x % 10)
    print("   " + hundreds)
    print("   " + tens)
    print("   " + units)

def visualize(clay, min_x, max_x, min_y, max_y):
    """
    Renders the grid with clay, sand, and the water spring.

    Args:
        clay (set of tuples): Set of (x, y) positions occupied by clay.
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
        min_y (int): Minimum y-coordinate to display.
        max_y (int): Maximum y-coordinate to display.
    """
    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Starting at y=0
    grid_max_y = max_y

    # Print x-axis labels
    print_x_labels(grid_min_x, grid_max_x)

    # Render each row of the grid
    for y in range(grid_min_y, grid_max_y + 1):
        line = ""
        for x in range(grid_min_x, grid_max_x + 1):
            if (x, y) == (500, 0):
                line += '+'  # Water spring
            elif (x, y) in clay:
                line += '#'  # Clay
            else:
                line += '.'  # Sand
        print(f"{y:>2} {line}")  # Right-align y for better formatting

def main():
    # Example input representing clay veins
    # input_data = [
    #     "x=495, y=2..7",
    #     "y=7, x=495..501",
    #     "x=501, y=3..7",
    #     "x=498, y=2..4",
    #     "x=506, y=1..2",
    #     "x=498, y=10..13",
    #     "x=504, y=10..13",
    #     "y=13, x=498..504"
    # ]
    input_data=["y=6, x=490..510"]
    # Parse the input to extract clay positions and grid boundaries
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Visualize the grid
    visualize(clay, min_x, max_x, min_y, max_y)

if __name__ == "__main__":
    main()

   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 .......................
 2 .......................
 3 .......................
 4 .......................
 5 .......................
 6 .#####################.


In [3]:
def add_water(input_data, water_units):
    """
    Simulates water flow in the grid and visualizes the result.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after adding water.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Starting at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    # Simulate water flow
    current_water = [(spring_x, 0)]
    for _ in range(water_units):
        if not current_water:
            break
        new_water = []
        for x, y in current_water:
            if y + 1 > grid_max_y or grid[y + 1][x] in ('#', '~'):
                # Water can't go further down; it spreads or settles
                if grid[y][x] == '.':
                    grid[y][x] = '~'  # Settled water
            else:
                if grid[y + 1][x] == '.':
                    grid[y + 1][x] = '|'
                    new_water.append((x, y + 1))
        current_water = new_water

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    add_water(input_data, 1)

   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 .......................
 3 .......................
 4 .......................
 5 .......................
 6 .#####################.


In [4]:
def simulate_water_flow(input_data, water_units):
    """
    Simulates water flow and visualizes the result after a given number of water units.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_down(x, y):
        """Flow water down until blocked or out of bounds."""
        while y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
            y += 1
            grid[y][x] = '|'
        return x, y

    def spread_water(x, y):
        """Spread water left and right, settling if constrained by clay."""
        left, right = x, x
        # Spread left
        while left - 1 >= grid_min_x and grid[y][left - 1] in ('.', '|') and grid[y + 1][left - 1] in ('#', '~'):
            left -= 1
            grid[y][left] = '|'
        # Spread right
        while right + 1 <= grid_max_x and grid[y][right + 1] in ('.', '|') and grid[y + 1][right + 1] in ('#', '~'):
            right += 1
            grid[y][right] = '|'

        # Check if water can settle
        if grid[y + 1][left] in ('#', '~') and grid[y + 1][right] in ('#', '~'):
            for x in range(left, right + 1):
                grid[y][x] = '~'
            return True  # Water settled
        return False  # Water did not settle

    # Simulate water flow
    water_sources = [(spring_x, 0)]
    for _ in range(water_units):
        if not water_sources:
            break
        new_sources = []
        for x, y in water_sources:
            # Flow down until blocked
            x, y = flow_down(x, y)
            if y + 1 > max_y:  # Out of bounds
                continue
            # Spread water left and right
            if not spread_water(x, y):
                new_sources.append((x, y))  # Continue from current position
        water_sources = new_sources

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_water_flow(input_data, 1)


   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........|...........
 5 ...........~~~~~~~~~~~.
 6 .#####################.


In [5]:
def simulate_single_water_unit(input_data, water_units):
    """
    Simulates a single unit of water flowing through the grid.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_single_unit(x, y):
        """
        Simulates the movement of a single water unit.
        Water moves down until blocked, then tries to spread.
        """
        # Move down until blocked
        while y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
            y += 1
            grid[y][x] = '|'
        # If water hits a block, mark it as settled
        if y + 1 > max_y or grid[y + 1][x] in ('#', '~'):
            grid[y][x] = '~'

    # Simulate water units
    for _ in range(water_units):
        flow_single_unit(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_single_water_unit(input_data, 1)


   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........|...........
 5 ...........~...........
 6 .#####################.


In [6]:
def simulate_single_water_unit(input_data, water_units):
    """
    Simulates a single unit of water flowing through the grid.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_single_unit(x, y):
        """
        Simulates the movement of a single water unit.
        Water moves down until blocked, then tries to spread.
        """
        # Move down until blocked
        while y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
            y += 1
            grid[y][x] = '|'
        # If water hits a block, mark it as settled
        if y + 1 > max_y or grid[y + 1][x] in ('#', '~'):
            grid[y][x] = '~'

    # Simulate water units
    for _ in range(water_units):
        flow_single_unit(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_single_water_unit(input_data, 2)


   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........~...........
 5 ...........~...........
 6 .#####################.


In [7]:
def simulate_water_units(input_data, water_units):
    """
    Simulates water flow through the grid for a specified number of units.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_water(x, y):
        """
        Simulates the movement of one water unit.
        Water flows down, then left or right if blocked.
        """
        # Move down until blocked or the bottom
        while y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
            y += 1
            grid[y][x] = '|'

        # If water is blocked below, try spreading left first
        if y + 1 > max_y or grid[y + 1][x] in ('#', '~'):
            # Spread left
            left = x
            while left - 1 >= grid_min_x and grid[y][left - 1] == '.' and grid[y + 1][left - 1] in ('#', '~'):
                left -= 1
                grid[y][left] = '~'
            # Spread right
            right = x
            while right + 1 <= grid_max_x and grid[y][right + 1] == '.' and grid[y + 1][right + 1] in ('#', '~'):
                right += 1
                grid[y][right] = '~'

            # Mark the current position as settled
            grid[y][x] = '~'

    # Simulate water units
    for _ in range(water_units):
        flow_water(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_water_units(input_data, 2)


   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........~~~~~~~~~~~.
 5 ...........~~~~~~~~~~~.
 6 .#####################.


In [8]:
def simulate_water_units(input_data, water_units):
    """
    Simulates water flow through the grid for a specified number of units.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_water(x, y):
        """
        Simulates the movement of one water unit.
        Water flows down, then spreads left and right as needed.
        """
        path = []  # Track path of flowing water

        # Flow down until blocked
        while y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
            y += 1
            path.append((x, y))
            if grid[y][x] != '~':  # Avoid overwriting settled water
                grid[y][x] = '|'

        # If water is blocked below, try spreading
        if y + 1 > max_y or grid[y + 1][x] in ('#', '~'):
            # Mark the current position as flowing
            grid[y][x] = '|'
            # Spread left first
            left = x
            while left - 1 >= grid_min_x and grid[y][left - 1] == '.' and grid[y + 1][left - 1] in ('#', '~'):
                left -= 1
                grid[y][left] = '~'
            # Spread right
            right = x
            while right + 1 <= grid_max_x and grid[y][right + 1] == '.' and grid[y + 1][right + 1] in ('#', '~'):
                right += 1
                grid[y][right] = '~'
            # Mark the entire row as settled if constrained
            if grid[y + 1][left] in ('#', '~') and grid[y + 1][right] in ('#', '~'):
                for xx in range(left, right + 1):
                    grid[y][xx] = '~'

        # Convert all flowing water in the path to settled water for this unit
        for px, py in path:
            if grid[py][px] != '~':
                grid[py][px] = '|'

    # Simulate water units
    for _ in range(water_units):
        flow_water(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_water_units(input_data, 2)

   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........~~~~~~~~~~~.
 5 ...........~~~~~~~~~~~.
 6 .#####################.


In [9]:
def simulate_water_units(input_data, water_units):
    """
    Simulates water flow through the grid for a specified number of units.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_water(x, y):
        """
        Simulates the movement of one water unit.
        Water flows down and spreads as needed.
        """
        while True:
            # Flow down
            if y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
                y += 1
                grid[y][x] = '|'
            # If blocked below, try spreading
            elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
                # Spread left
                left = x
                while left - 1 >= grid_min_x and grid[y][left - 1] == '.' and grid[y + 1][left - 1] in ('#', '~'):
                    left -= 1
                    grid[y][left] = '|'
                # Spread right
                right = x
                while right + 1 <= grid_max_x and grid[y][right + 1] == '.' and grid[y + 1][right + 1] in ('#', '~'):
                    right += 1
                    grid[y][right] = '|'

                # If fully constrained, settle water
                if grid[y + 1][left] in ('#', '~') and grid[y + 1][right] in ('#', '~'):
                    for xx in range(left, right + 1):
                        grid[y][xx] = '~'
                return  # Stop simulation for this unit
            else:
                return  # Stop if out of bounds or no further movement

    # Simulate water units
    for _ in range(water_units):
        flow_water(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_water_units(input_data, 2)

   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........~~~~~~~~~~~.
 5 ...........~~~~~~~~~~~.
 6 .#####################.


In [10]:
def simulate_water_units(input_data, water_units):
    """
    Simulates water flow through the grid for a specified number of units.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_water(x, y):
        """
        Simulates the movement of one water unit following the described logic.
        """
        # Move down until blocked or out of bounds
        while y + 1 <= max_y and grid[y + 1][x] == '.':
            y += 1
            grid[y][x] = '|'

        # If blocked below, place settled water (~)
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            grid[y][x] = '~'

            # Try to flow left if possible
            left = x - 1
            while left >= grid_min_x and grid[y][left] == '.':
                # If there's clay or settled water below, mark as settled (~)
                if grid[y + 1][left] in ('#', '~'):
                    grid[y][left] = '~'
                else:
                    grid[y][left] = '|'
                left -= 1

    # Simulate water units
    for _ in range(water_units):
        flow_water(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_water_units(input_data, 2)


   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........|...........
 5 ...........~...........
 6 .#####################.


In [12]:
def simulate_water_units(input_data, water_units):
    """
    Simulates water flow through the grid for a specified number of units.

    Args:
        input_data (list of str): Input data defining the clay veins.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the visualization after simulating water flow.
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Start at y=0
    grid_max_y = max_y

    # Initialize the grid with sand
    grid = [['.' for _ in range(grid_min_x, grid_max_x + 1)] for _ in range(grid_min_y, grid_max_y + 1)]

    # Add clay to the grid
    for x, y in clay:
        grid[y][x - grid_min_x] = '#'

    # Add the water spring
    spring_x = 500 - grid_min_x
    grid[0][spring_x] = '+'

    def flow_water(x, y):
        """
        Simulates the movement of one water unit following the described logic.
        """
        # Move down until blocked or out of bounds
        while y + 1 <= max_y and grid[y + 1][x] in ('.', '|'):
            y += 1
            if grid[y][x] == '.':  # Only overwrite sand
                grid[y][x] = '|'

        # If blocked below, place settled water (~)
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            if grid[y][x] == '.':  # Only overwrite sand
                grid[y][x] = '~'

            # Try to flow left if possible
            left = x - 1
            while left >= grid_min_x and grid[y][left] == '.':
                # If there's clay or settled water below, mark as settled (~)
                if grid[y + 1][left] in ('#', '~'):
                    grid[y][left] = '~'
                else:
                    grid[y][left] = '|'
                left -= 1

    # Simulate water units
    for _ in range(water_units):
        flow_water(spring_x, 0)

    # Visualize the grid
    print_x_labels(grid_min_x, grid_max_x)
    for y, row in enumerate(grid):
        print(f"{y:>2} {''.join(row)}")


# Example usage
if __name__ == "__main__":
    input_data = ["y=6, x=490..510"]
    simulate_water_units(input_data, 2)


   44444444444555555555555
   89999999999000000000011
   90123456789012345678901
 0 ...........+...........
 1 ...........|...........
 2 ...........|...........
 3 ...........|...........
 4 ...........|...........
 5 ...........|...........
 6 .#####################.


In [14]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x, y_range = line[2:].split(", y=")
            x = int(x)
            y_start, y_end = map(int, y_range.split(".."))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith("y="):
            y, x_range = line[2:].split(", x=")
            y = int(y)
            x_start, x_end = map(int, x_range.split(".."))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    min_y = min(y for x, y in clay)
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x, max_y

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()

def simulate_water_flow(grid, spring=(500, 0), min_x=0, max_y=0):
    water = [(spring[0] - min_x, spring[1])]
    reached = set()
    
    while water:
        new_water = []
        for x, y in water:
            if y > max_y:  # Ignore water below the max depth
                continue
            
            # Flow downward
            if y + 1 <= max_y and grid[y + 1, x] == '.':
                grid[y + 1, x] = '|'
                new_water.append((x, y + 1))
            else:
                # Check spreading left and right
                spread_left = x - 1
                while grid[y, spread_left] == '.' and grid[y + 1, spread_left] in ('#', '~'):
                    grid[y, spread_left] = '|'
                    new_water.append((spread_left, y))
                    spread_left -= 1

                spread_right = x + 1
                while grid[y, spread_right] == '.' and grid[y + 1, spread_right] in ('#', '~'):
                    grid[y, spread_right] = '|'
                    new_water.append((spread_right, y))
                    spread_right += 1

                # Settle water
                if grid[y, spread_left] == '#' and grid[y, spread_right] == '#':
                    for sx in range(spread_left + 1, spread_right):
                        grid[y, sx] = '~'
                        reached.add((sx, y))
        
        water = new_water
    
    return reached

def count_reached_tiles(grid, min_y, max_y):
    return sum(
        1
        for y in range(min_y, max_y + 1)
        for x in range(grid.shape[1])
        if grid[y, x] in ('|', '~')
    )

# Input
clay_lines = [
"y=6, x=490..510"
]

# Simulation
clay = parse_input(clay_lines)
grid, min_x, max_y = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

spring = (500, 0)
reached_tiles = simulate_water_flow(grid, spring, min_x, max_y)

print("Final Grid:")
print_grid(grid)

total_reached = count_reached_tiles(grid, min([y for _, y in clay]), max_y)
print(f"Total tiles reached by water: {total_reached}")


Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

Final Grid:
...........+...........
...........|...........
...........|...........
...........|...........
...........|...........
.|||||||||||||||||||||.
.#####################.

Total tiles reached by water: 0


In [21]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x, y_range = line[2:].split(", y=")
            x = int(x)
            y_start, y_end = map(int, y_range.split(".."))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith("y="):
            y, x_range = line[2:].split(", x=")
            y = int(y)
            x_start, x_end = map(int, x_range.split(".."))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()

def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]
    unit_count = 0

    while unit_count < num_units and water:
        new_water = []
        for x, y in water:
            # Flow downward
            if y + 1 <= max_y and grid[y + 1, x] == '.':
                grid[y + 1, x] = '|'
                new_water.append((x, y + 1))
            elif y + 1 <= max_y and grid[y + 1, x] in ('#', '~'):
                # Spread left
                spread_left = x - 1
                while grid[y, spread_left] == '.' and grid[y + 1, spread_left] in ('#', '~'):
                    grid[y, spread_left] = '|'
                    new_water.append((spread_left, y))
                    spread_left -= 1

                # Spread right
                spread_right = x + 1
                while grid[y, spread_right] == '.' and grid[y + 1, spread_right] in ('#', '~'):
                    grid[y, spread_right] = '|'
                    new_water.append((spread_right, y))
                    spread_right += 1

                # Settle if enclosed
                if grid[y, spread_left] == '#' and grid[y, spread_right] == '#':
                    for sx in range(spread_left + 1, spread_right):
                        grid[y, sx] = '~'

        unit_count += 1
        water = new_water
        print(f"After {unit_count} unit(s):")
        print_grid(grid)
    return grid

# Input
clay_lines = ["y=6, x=490..510"]
num_water_units = 3

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 7, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)

Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

After 1 unit(s):
...........+...........
...........|...........
.......................
.......................
.......................
.......................
.#####################.

After 2 unit(s):
...........+...........
...........|...........
...........|...........
.......................
.......................
.......................
.#####################.

After 3 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
.......................
.......................
.#####################.

After 4 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
...........|...........
.......................
.#####################.

After 5 unit(s):
...........+...........
...........|...........
........

In [22]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x, y_range = line[2:].split(", y=")
            x = int(x)
            y_start, y_end = map(int, y_range.split(".."))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith("y="):
            y, x_range = line[2:].split(", x=")
            y = int(y)
            x_start, x_end = map(int, x_range.split(".."))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()

def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]
    unit_count = 0

    while unit_count < num_units and water:
        new_water = []
        for x, y in water:
            # Flow downward
            if y + 1 <= max_y and grid[y + 1, x] == '.':
                grid[y + 1, x] = '|'
                new_water.append((x, y + 1))
            else:
                # Spread left and right
                spread_left = x
                while grid[y, spread_left - 1] == '.' and grid[y + 1, spread_left - 1] in ('#', '~'):
                    spread_left -= 1
                    grid[y, spread_left] = '|'

                spread_right = x
                while grid[y, spread_right + 1] == '.' and grid[y + 1, spread_right + 1] in ('#', '~'):
                    spread_right += 1
                    grid[y, spread_right] = '|'

                # Check for settling
                if grid[y, spread_left - 1] == '#' and grid[y, spread_right + 1] == '#':
                    for sx in range(spread_left, spread_right + 1):
                        grid[y, sx] = '~'

        unit_count += 1
        water = new_water
        print(f"After {unit_count} unit(s):")
        print_grid(grid)
    return grid

# Input
clay_lines = ["y=6, x=490..510"]
num_water_units = 5

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 7, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

After 1 unit(s):
...........+...........
...........|...........
.......................
.......................
.......................
.......................
.#####################.

After 2 unit(s):
...........+...........
...........|...........
...........|...........
.......................
.......................
.......................
.#####################.

After 3 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
.......................
.......................
.#####################.

After 4 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
...........|...........
.......................
.#####################.

After 5 unit(s):
...........+...........
...........|...........
........

In [24]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x, y_range = line[2:].split(", y=")
            x = int(x)
            y_start, y_end = map(int, y_range.split(".."))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith("y="):
            y, x_range = line[2:].split(", x=")
            y = int(y)
            x_start, x_end = map(int, x_range.split(".."))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()

def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]
    unit_count = 0

    while unit_count < num_units and water:
        new_water = []
        for x, y in water:
            # Flow downward
            if y + 1 <= max_y and grid[y + 1, x] == '.':
                grid[y + 1, x] = '|'
                new_water.append((x, y + 1))
            # Settle if blocked below
            elif y + 1 <= max_y and grid[y + 1, x] in ('#', '~'):
                grid[y, x] = '~'

            # Spread left and right if blocked below
            else:
                # Spread left
                spread_left = x
                while grid[y, spread_left - 1] == '.' and grid[y + 1, spread_left - 1] in ('#', '~'):
                    spread_left -= 1
                    grid[y, spread_left] = '|'

                # Spread right
                spread_right = x
                while grid[y, spread_right + 1] == '.' and grid[y + 1, spread_right + 1] in ('#', '~'):
                    spread_right += 1
                    grid[y, spread_right] = '|'

                # Settle if enclosed
                if grid[y, spread_left - 1] == '#' and grid[y, spread_right + 1] == '#':
                    for sx in range(spread_left, spread_right + 1):
                        grid[y, sx] = '~'

        unit_count += 1
        water = new_water
        print(f"After {unit_count} unit(s):")
        print_grid(grid)
    return grid

# Input
clay_lines = ["y=6, x=490..510"]
num_water_units = 5

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 8, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

After 1 unit(s):
...........+...........
...........|...........
.......................
.......................
.......................
.......................
.#####################.

After 2 unit(s):
...........+...........
...........|...........
...........|...........
.......................
.......................
.......................
.#####################.

After 3 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
.......................
.......................
.#####################.

After 4 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
...........|...........
.......................
.#####################.

After 5 unit(s):
...........+...........
...........|...........
........

In [25]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x, y_range = line[2:].split(", y=")
            x = int(x)
            y_start, y_end = map(int, y_range.split(".."))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith("y="):
            y, x_range = line[2:].split(", x=")
            y = int(y)
            x_start, x_end = map(int, x_range.split(".."))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()

def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]
    unit_count = 0

    while unit_count < num_units and water:
        new_water = []
        for x, y in water:
            # Flow downward
            if y + 1 <= max_y and grid[y + 1, x] == '.':
                grid[y + 1, x] = '|'
                new_water.append((x, y + 1))
            # Settle if blocked below
            elif y + 1 <= max_y and grid[y + 1, x] in ('#', '~'):
                grid[y, x] = '~'

            # Stop flowing further downward if settled
            else:
                continue

        unit_count += 1
        water = new_water
        print(f"After {unit_count} unit(s):")
        print_grid(grid)
    return grid

# Input
clay_lines = ["y=6, x=490..510"]
num_water_units = 6

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, num_water_units, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

After 1 unit(s):
...........+...........
...........|...........
.......................
.......................
.......................
.......................
.#####################.

After 2 unit(s):
...........+...........
...........|...........
...........|...........
.......................
.......................
.......................
.#####################.

After 3 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
.......................
.......................
.#####################.

After 4 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
...........|...........
.......................
.#####################.

After 5 unit(s):
...........+...........
...........|...........
........

In [47]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x, y_range = line[2:].split(", y=")
            x = int(x)
            y_start, y_end = map(int, y_range.split(".."))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith("y="):
            y, x_range = line[2:].split(", x=")
            y = int(y)
            x_start, x_end = map(int, x_range.split(".."))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()


def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]
    unit_count = 0

    while unit_count < num_units:
        new_water = []
        for x, y in water:
            print('water')
            print(water)
            print('water')
            # Check two rows below
            if y + 2 <= max_y:
                # If two rows below is sand (.), flow downward
                if grid[y + 2, x] == '.':
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '|'
                        new_water.append((x, y + 1))
                # If two rows below is a barrier, settle the cell below
                elif grid[y + 2, x] in ('#', '~'):
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '~'

            # If the cell below is a barrier (#)
            elif y + 1 <= max_y and grid[y + 1, x] == '#':
                # Check the cell to the left
                
                if x - 1 >= 0 and grid[y, x - 1] == '.':
                    
                    # Look below the left cell
                    if y + 1 <= max_y and grid[y + 1, x - 1] == '#':
                        grid[y, x - 1] = '~'  # Settle the left cell
                    elif y + 1 <= max_y and grid[y + 1, x - 1] == '.':
                        grid[y, x - 1] = '|'  # Allow flowing water
                        new_water.append((x - 1, y))

            # If the cell cannot flow further, retain the current water
            if grid[y, x] == '|':
                new_water.append((x, y))

        # Increment the unit count and ensure continuation
        unit_count += 1
        water = new_water if new_water else water  # Continue if no new water is added
        print(f"After {unit_count} unit(s):")
        print_grid(grid)
    return grid



# Input
clay_lines = ["y=6, x=490..510"]
num_water_units = 6

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 8, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

water
[(11, 0)]
water
After 1 unit(s):
...........+...........
...........|...........
.......................
.......................
.......................
.......................
.#####################.

water
[(11, 1)]
water
After 2 unit(s):
...........+...........
...........|...........
...........|...........
.......................
.......................
.......................
.#####################.

water
[(11, 2), (11, 1)]
water
water
[(11, 2), (11, 1)]
water
After 3 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
.......................
.......................
.#####################.

water
[(11, 3), (11, 2), (11, 1)]
water
water
[(11, 3), (11, 2), (11, 1)]
water
water
[(11, 3), (11, 2), (11, 1)]
water
After 4 unit(s):
...........+....

In [61]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x_range, y_range = line[2:].split(", y=")
            if ".." in x_range:
                x_start, x_end = map(int, x_range.split(".."))
            else:
                x_start = x_end = int(x_range)
            if ".." in y_range:
                y_start, y_end = map(int, y_range.split(".."))
            else:
                y_start = y_end = int(y_range)
            for x in range(x_start, x_end + 1):
                for y in range(y_start, y_end + 1):
                    clay.add((x, y))
        elif line.startswith("y="):
            y_range, x_range = line[2:].split(", x=")
            if ".." in y_range:
                y_start, y_end = map(int, y_range.split(".."))
            else:
                y_start = y_end = int(y_range)
            if ".." in x_range:
                x_start, x_end = map(int, x_range.split(".."))
            else:
                x_start = x_end = int(x_range)
            for x in range(x_start, x_end + 1):
                for y in range(y_start, y_end + 1):
                    clay.add((x, y))
    return clay


def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()


def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]  # Initialize with the spring position
    unit_count = 0

    while unit_count < num_units and water:
        new_water = set()  # Use a set to avoid duplicates
        to_remove = []  # Track cells that no longer need to be revisited
        for x, y in water:
            # Check two rows below
            if y + 2 <= max_y:
                # If two rows below is sand (.), flow downward
                if grid[y + 2, x] == '.':
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '|'
                        new_water.add((x, y + 1))
                    to_remove.append((x, y))
                # If two rows below is a barrier, settle the cell below
                elif grid[y + 2, x] in ('#', '~'):
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '~'
                        new_water.add((x, y + 1))
                        to_remove.append((x, y))

            # If the cell below is a barrier (#)
            elif y + 1 <= max_y and grid[y + 1, x] in ('#', '~'):
                # Check the cell to the left
                if x - 1 >= 0 and grid[y, x - 1] == '.':
                    # Look below the left cell
                    if y + 1 <= max_y and grid[y + 1, x - 1] == '#':
                        grid[y, x - 1] = '~'  # Settle the left cell
                        new_water.add((x - 1, y))
                    elif y + 1 <= max_y and grid[y + 1, x - 1] == '.':
                        grid[y, x - 1] = '|'  # Allow flowing water
                        new_water.add((x - 1, y))
                    # Once the left cell is processed, move on
                    to_remove.append((x, y))

            # If the current cell is still flowing
            if grid[y, x] == '|':
                new_water.add((x, y))

        # Update the water list by removing processed cells
        water = [w for w in water if w not in to_remove] + list(new_water)

        # Increment the unit count and print the grid
        unit_count += 1
        print(f"After {unit_count} unit(s):")
        print_grid(grid)

    return grid





# Input
clay_lines = ["y=6, x=490..510"]
num_water_units = 6

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 14, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
...........+...........
.......................
.......................
.......................
.......................
.......................
.#####################.

After 1 unit(s):
...........+...........
...........|...........
.......................
.......................
.......................
.......................
.#####################.

After 2 unit(s):
...........+...........
...........|...........
...........|...........
.......................
.......................
.......................
.#####################.

After 3 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
.......................
.......................
.#####################.

After 4 unit(s):
...........+...........
...........|...........
...........|...........
...........|...........
...........|...........
.......................
.#####################.

After 5 unit(s):
...........+...........
...........|...........
........

In [66]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x_range, y_range = line[2:].split(", y=")
            if ".." in x_range:  # Handle x range
                x_start, x_end = map(int, x_range.split(".."))
            else:
                x_start = x_end = int(x_range)
            if ".." in y_range:  # Handle y range
                y_start, y_end = map(int, y_range.split(".."))
            else:
                y_start = y_end = int(y_range)
            for x in range(x_start, x_end + 1):
                for y in range(y_start, y_end + 1):
                    clay.add((x, y))
        elif line.startswith("y="):
            y_range, x_range = line[2:].split(", x=")
            if ".." in y_range:  # Handle y range
                y_start, y_end = map(int, y_range.split(".."))
            else:
                y_start = y_end = int(y_range)
            if ".." in x_range:  # Handle x range
                x_start, x_end = map(int, x_range.split(".."))
            else:
                x_start = x_end = int(x_range)
            for x in range(x_start, x_end + 1):
                for y in range(y_start, y_end + 1):
                    clay.add((x, y))
    return clay


def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()


def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]  # Initialize with the spring position
    unit_count = 0

    while unit_count < num_units and water:
        new_water = set()  # Use a set to avoid duplicates
        to_remove = []  # Track cells that no longer need to be revisited
        for x, y in water:
            print(f'x, y is {x}, {y}, water is {water}, grid value is {grid[x,y]}')
            # Check two rows below
            if y + 2 <= max_y:
                # If two rows below is sand (.), flow downward
                if grid[y + 2, x] == '.':
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '|'
                        new_water.add((x, y + 1))
                    to_remove.append((x, y))
                # If two rows below is a barrier, settle the cell below
                elif grid[y + 2, x] in ('#', '~'):
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '~'
                        new_water.add((x, y + 1))
                        to_remove.append((x, y))

            # If the cell below is a barrier (#)
            elif y + 1 <= max_y and grid[y + 1, x] in ('#', '~'):
                # Check the cell to the left
                if x - 1 >= 0 and grid[y, x - 1] == '.':
                    # Look below the left cell
                    if y + 1 <= max_y and grid[y + 1, x - 1] == '#':
                        grid[y, x - 1] = '~'  # Settle the left cell
                        new_water.add((x - 1, y))
                    elif y + 1 <= max_y and grid[y + 1, x - 1] == '.':
                        grid[y, x - 1] = '|'  # Allow flowing water
                        new_water.add((x - 1, y))
                    # Once the left cell is processed, move on
                    to_remove.append((x, y))

                # Check the cell to the right (new logic)
                if x + 1 < max_x and grid[y, x + 1] == '.':
                    # Look below the right cell
                    print(testing)
                    if y + 1 <= max_y and grid[y + 1, x + 1] == '#':
                        grid[y, x + 1] = '~'  # Settle the right cell
                        new_water.add((x + 1, y))
                    elif y + 1 <= max_y and grid[y + 1, x + 1] == '.':
                        grid[y, x + 1] = '|'  # Allow flowing water
                        new_water.add((x + 1, y))
                    # Once the right cell is processed, move on
                    to_remove.append((x, y))

            # If the current cell is still flowing
            if grid[y, x] == '|':
                new_water.add((x, y))

        # Update the water list by removing processed cells
        water = [w for w in water if w not in to_remove] + list(new_water)

        # Increment the unit count and print the grid
        unit_count += 1
        print(f"After {unit_count} unit(s):")
        print_grid(grid)

    return grid






# Input
clay_lines = [
    "x=499, y=0..1",  # Vertical clay from (499, 0) to (499, 1)
    "x=499..500, y=2",  # Horizontal clay at y=2, from (499, 2) to (500, 2)
    "x=499..502, y=3"   # Horizontal clay at y=3, from (499, 3) to (502, 3)
]
num_water_units = 6

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 14, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
.#+...
.#....
.##...
.####.

x, y is 2, 0, water is [(2, 0)], grid value is .
After 1 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 2 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 3 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 4 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 5 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 6 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 7 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 8 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 9 unit(s):
.#+...
.#~...
.##...
.####.

x, y is 2, 1, water is [(2, 1)], grid value is #
After 10 unit(s):
.#+...
.#~...
.##...
.####.

x, y i

In [68]:
import numpy as np

def parse_input(clay_lines):
    clay = set()
    for line in clay_lines:
        if line.startswith("x="):
            x_range, y_range = line[2:].split(", y=")
            if ".." in x_range:
                x_start, x_end = map(int, x_range.split(".."))
            else:
                x_start = x_end = int(x_range)
            if ".." in y_range:
                y_start, y_end = map(int, y_range.split(".."))
            else:
                y_start = y_end = int(y_range)
            for x in range(x_start, x_end + 1):
                for y in range(y_start, y_end + 1):
                    clay.add((x, y))
        elif line.startswith("y="):
            y_range, x_range = line[2:].split(", x=")
            if ".." in y_range:
                y_start, y_end = map(int, y_range.split(".."))
            else:
                y_start = y_end = int(y_range)
            if ".." in x_range:
                x_start, x_end = map(int, x_range.split(".."))
            else:
                x_start = x_end = int(x_range)
            for x in range(x_start, x_end + 1):
                for y in range(y_start, y_end + 1):
                    clay.add((x, y))
    return clay


def create_grid(clay, spring=(500, 0)):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    
    grid = np.full((max_y + 1, max_x - min_x + 1), '.', dtype=str)
    for x, y in clay:
        grid[y, x - min_x] = '#'
    grid[spring[1], spring[0] - min_x] = '+'
    
    return grid, min_x

def print_grid(grid):
    for row in grid:
        print("".join(row))
    print()


def simulate_water_flow(grid, num_units, spring=(500, 0), min_x=0):
    max_y, max_x = grid.shape
    max_y -= 1
    spring_x = spring[0] - min_x
    water = [(spring_x, spring[1])]  # Initialize with the spring position
    unit_count = 0

    while unit_count < num_units and water:
        new_water = set()  # Use a set to avoid duplicates
        to_remove = []  # Track cells that no longer need to be revisited
        for x, y in water:
            # Check two rows below
            if y + 2 <= max_y:
                # If two rows below is sand (.), flow downward
                if grid[y + 2, x] == '.':
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '|'
                        new_water.add((x, y + 1))
                    to_remove.append((x, y))
                # If two rows below is a barrier, settle the cell below
                elif grid[y + 2, x] in ('#', '~'):
                    if grid[y + 1, x] == '.':
                        grid[y + 1, x] = '~'
                        new_water.add((x, y + 1))
                        to_remove.append((x, y))

            # If the cell below is a barrier (#)
            elif y + 1 <= max_y and grid[y + 1, x] in ('#', '~'):
                # Check the cell to the left
                if x - 1 >= 0 and grid[y, x - 1] == '.':
                    # Look below the left cell
                    if y + 1 <= max_y and grid[y + 1, x - 1] == '#':
                        grid[y, x - 1] = '~'  # Settle the left cell
                        new_water.add((x - 1, y))
                    elif y + 1 <= max_y and grid[y + 1, x - 1] == '.':
                        grid[y, x - 1] = '|'  # Allow flowing water
                        new_water.add((x - 1, y))
                    # Once the left cell is processed, move on
                    to_remove.append((x, y))

            # If the current cell is still flowing
            if grid[y, x] == '|':
                new_water.add((x, y))

        # Update the water list by removing processed cells
        water = [w for w in water if w not in to_remove] + list(new_water)

        # Increment the unit count and print the grid
        unit_count += 1
        print(f"After {unit_count} unit(s):")
        print_grid(grid)

    return grid





# Input
clay_lines =    [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]
num_water_units = 6

# Simulation
clay = parse_input(clay_lines)
grid, min_x = create_grid(clay)
print("Initial Grid:")
print_grid(grid)

# Simulate flow
final_grid = simulate_water_flow(grid, 14, spring=(500, 0), min_x=min_x)
print("Final Grid:")
print_grid(final_grid)


Initial Grid:
......+.......
............#.
.#..#.......#.
.#..#..#......
.#..#..#......
.#.....#......
.#.....#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...

After 1 unit(s):
......+.......
......|.....#.
.#..#.......#.
.#..#..#......
.#..#..#......
.#.....#......
.#.....#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...

After 2 unit(s):
......+.......
......|.....#.
.#..#.|.....#.
.#..#..#......
.#..#..#......
.#.....#......
.#.....#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...

After 3 unit(s):
......+.......
......|.....#.
.#..#.|.....#.
.#..#.|#......
.#..#..#......
.#.....#......
.#.....#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...

After 4 unit(s):
......+.......
......|.....#.
.#..#.|.....#.
.#..#.|#......
.#..#.|#......

In [69]:
import re
from collections import deque

# Parse input
def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

# Initialize grid
def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

# Display the grid
def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

# Simulate water flow
def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue
        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
        elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            # Spread left and right
            left, right = x, x
            while grid[y][left - 1] == '.' and grid[y + 1][left - 1] in ('#', '~'):
                left -= 1
            while grid[y][right + 1] == '.' and grid[y + 1][right + 1] in ('#', '~'):
                right += 1
            enclosed = grid[y][left - 1] == '#' and grid[y][right + 1] == '#'
            for i in range(left, right + 1):
                grid[y][i] = '~' if enclosed else '|'
            if not enclosed:
                if grid[y][left - 1] == '.':
                    queue.append((left - 1, y))
                if grid[y][right + 1] == '.':
                    queue.append((right + 1, y))
            else:
                queue.append((x, y - 1))

# Main function
def main(input_data):
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)

# Example usage
input_data = """\
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""
main(input_data)


......+.......
......|.....#.
.#..#|||....#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
....|||||||...
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..



In [70]:
import re
from collections import deque

# Parse input
def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

# Initialize grid
def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

# Display the grid
def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

# Simulate water flow
def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue
        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
        elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            # Spread left and right
            left, right = x, x
                # Spread left
            while grid[y][left - 1] == '.':
                if grid[y + 1][left - 1] in ('#', '~'):
                    left -= 1
                else:
                    break
            
            # Spread right
            while grid[y][right + 1] == '.':
                if grid[y + 1][right + 1] in ('#', '~'):
                    right += 1
                else:
                    break

            enclosed = grid[y][left - 1] == '#' and grid[y][right + 1] == '#'
            for i in range(left, right + 1):
                grid[y][i] = '~' if enclosed else '|'
            if not enclosed:
                if grid[y][left - 1] == '.':
                    queue.append((left - 1, y))
                if grid[y][right + 1] == '.':
                    queue.append((right + 1, y))
            else:
                queue.append((x, y - 1))

# Main function
def main(input_data):
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)

# Example usage
input_data = """\
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""
main(input_data)


......+.......
......|.....#.
.#..#|||....#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
....|||||||...
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..



In [71]:
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue
        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
        elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            # Spread left and right
            left, right = x, x
            while grid[y][left - 1] == '.' and grid[y + 1][left - 1] in ('#', '~'):
                left -= 1
            while grid[y][right + 1] == '.' and grid[y + 1][right + 1] in ('#', '~'):
                right += 1
            enclosed = grid[y][left - 1] == '#' and grid[y][right + 1] == '#'
            for i in range(left, right + 1):
                grid[y][i] = '~' if enclosed else '|'
            if not enclosed:
                if grid[y][left - 1] == '.':
                    queue.append((left - 1, y))
                if grid[y][right + 1] == '.':
                    queue.append((right + 1, y))
            else:
                queue.append((x, y - 1))
        elif y + 1 <= max_y and grid[y + 1][x] == '|':
            queue.append((x, y + 1))

# Main function
def main(input_data):
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)

# Example usage
input_data = """\
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""
main(input_data)


......+.......
......|.....#.
.#..#|||....#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
....|||||||...
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..



In [72]:
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue
        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
        elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            # Spread left and right
            left, right = x, x
            left_fall, right_fall = False, False
            
            # Spread left
            while grid[y][left - 1] == '.' or (grid[y][left - 1] == '|' and grid[y + 1][left - 1] == '.'):
                left -= 1
                if grid[y + 1][left] == '.':
                    left_fall = True
                    break
            
            # Spread right
            while grid[y][right + 1] == '.' or (grid[y][right + 1] == '|' and grid[y + 1][right + 1] == '.'):
                right += 1
                if grid[y + 1][right] == '.':
                    right_fall = True
                    break
            
            enclosed = not left_fall and not right_fall and grid[y][left - 1] == '#' and grid[y][right + 1] == '#'
            
            for i in range(left, right + 1):
                grid[y][i] = '~' if enclosed else '|'
            
            # Handle falling water
            if left_fall:
                queue.append((left, y))
            if right_fall:
                queue.append((right, y))
            
            # Backtrack upward if enclosed
            if enclosed:
                queue.append((x, y - 1))

# Main function
def main(input_data):
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)

# Example usage
input_data = """\
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""
main(input_data)


......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
...|||||||||..
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..



In [78]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Check if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            left_blocked, right_blocked = False, False
            
            # Spread left
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
                    queue.append((left, y))
                    break
            else:
                left_blocked = grid[y][left - 1] in ('#', '~')

            # Spread right
            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
                    queue.append((right, y))
                    break
            else:
                right_blocked = grid[y][right + 1] in ('#', '~')

            # If the row is enclosed, convert to settled water
            if left_blocked and right_blocked:
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling


def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))

# Main function
def main(input_data):
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
input_data = """\
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""
main(input_data)


......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
...|||||||||..
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..

Total water tiles (~ or |): 57


In [16]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

# def simulate_water(grid, min_x, max_y):
#     spring = (500 - min_x, 0)
#     grid[spring[1]][spring[0]] = '+'
#     queue = deque([spring])
#     while queue:
#         x, y = queue.popleft()
#         if y > max_y:
#             continue
#         # Flow downward
#         if y + 1 <= max_y and grid[y + 1][x] == '.':
#             grid[y + 1][x] = '|'
#             queue.append((x, y + 1))
#         elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
#             # Spread left and right
#             left, right = x, x
#             left_fall, right_fall = False, False
            
#             # Spread left
#             while grid[y][left - 1] == '.' or (grid[y][left - 1] == '|' and grid[y + 1][left - 1] == '.'):
#                 left -= 1
#                 if grid[y + 1][left] == '.':
#                     left_fall = True
#                     break
            
#             # Spread right
#             while grid[y][right + 1] == '.' or (grid[y][right + 1] == '|' and grid[y + 1][right + 1] == '.'):
#                 right += 1
#                 if grid[y + 1][right] == '.':
#                     right_fall = True
#                     break
            
#             enclosed = not left_fall and not right_fall and grid[y][left - 1] == '#' and grid[y][right + 1] == '#'
            
#             for i in range(left, right + 1):
#                 grid[y][i] = '~' if enclosed else '|'
            
#             # Handle falling water
#             if left_fall:
#                 queue.append((left, y))
#             if right_fall:
#                 queue.append((right, y))
            
#             # Backtrack upward if enclosed
#             if enclosed:
#                 queue.append((x, y - 1))
# def simulate_water(grid, min_x, max_y):
#     spring = (500 - min_x, 0)
#     grid[spring[1]][spring[0]] = '+'
#     queue = deque([spring])
    
#     while queue:
#         x, y = queue.popleft()
#         if y > max_y:
#             continue

#         # Flow downward
#         if y + 1 <= max_y and grid[y + 1][x] == '.':
#             grid[y + 1][x] = '|'
#             queue.append((x, y + 1))
#             continue

#         # Check if downward flow is blocked
#         if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
#             left, right = x, x
#             left_blocked, right_blocked = False, False
            
#             # Spread left
#             while grid[y][left - 1] == '.':
#                 left -= 1
#                 grid[y][left] = '|'
#                 if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
#                     queue.append((left, y))
#                     break
#             else:
#                 left_blocked = grid[y][left - 1] in ('#', '~')

#             # Spread right
#             while grid[y][right + 1] == '.':
#                 right += 1
#                 grid[y][right] = '|'
#                 if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
#                     queue.append((right, y))
#                     break
#             else:
#                 right_blocked = grid[y][right + 1] in ('#', '~')

#             # If the row is enclosed, convert to settled water
#             if left_blocked and right_blocked:
#                 for i in range(left, right + 1):
#                     grid[y][i] = '~'
#                 queue.append((x, y - 1))  # Recheck row above for settling
def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Spread left and right if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            left_blocked, right_blocked = False, False
            
            # Spread left
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
                    queue.append((left, y))
                    break
            else:
                left_blocked = grid[y][left - 1] in ('#', '~')

            # Spread right
            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
                    queue.append((right, y))
                    break
            else:
                right_blocked = grid[y][right + 1] in ('#', '~')

            # Handle joined falling streams
            if not left_blocked and not right_blocked:
                # If both ends are unblocked, let the water flow freely
                queue.append((left, y))
                queue.append((right, y))
                continue

            # If the row is enclosed, convert to settled water
            if left_blocked and right_blocked:
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling


def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))
def process_paired_rows(grid):
    """
    Processes rows with the pattern:
    #, followed by |, ending with #, where the row below is fully supportive.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 1):  # Stop at the second-to-last row
        row = grid[y]
        below_row = grid[y + 1]
        
        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a valid segment of | bounded by #
                while x < width and row[x] == '|':
                    x += 1
                
                if x < width and row[x] == '#':  # End boundary found
                    end = x
                    
                    # Verify the below row has matching boundaries
                    if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] == '#' for i in range(start + 1, end)):
                        # Replace | with ~ in the current row
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1
# Main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    process_paired_rows(grid)
    display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
if __name__ == "__main__":
    main('sample-input.txt')


......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
...|||||||||..
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..

Total water tiles (~ or |): 57


In [17]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

# def simulate_water(grid, min_x, max_y):
#     spring = (500 - min_x, 0)
#     grid[spring[1]][spring[0]] = '+'
#     queue = deque([spring])
#     while queue:
#         x, y = queue.popleft()
#         if y > max_y:
#             continue
#         # Flow downward
#         if y + 1 <= max_y and grid[y + 1][x] == '.':
#             grid[y + 1][x] = '|'
#             queue.append((x, y + 1))
#         elif y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
#             # Spread left and right
#             left, right = x, x
#             left_fall, right_fall = False, False
            
#             # Spread left
#             while grid[y][left - 1] == '.' or (grid[y][left - 1] == '|' and grid[y + 1][left - 1] == '.'):
#                 left -= 1
#                 if grid[y + 1][left] == '.':
#                     left_fall = True
#                     break
            
#             # Spread right
#             while grid[y][right + 1] == '.' or (grid[y][right + 1] == '|' and grid[y + 1][right + 1] == '.'):
#                 right += 1
#                 if grid[y + 1][right] == '.':
#                     right_fall = True
#                     break
            
#             enclosed = not left_fall and not right_fall and grid[y][left - 1] == '#' and grid[y][right + 1] == '#'
            
#             for i in range(left, right + 1):
#                 grid[y][i] = '~' if enclosed else '|'
            
#             # Handle falling water
#             if left_fall:
#                 queue.append((left, y))
#             if right_fall:
#                 queue.append((right, y))
            
#             # Backtrack upward if enclosed
#             if enclosed:
#                 queue.append((x, y - 1))
# def simulate_water(grid, min_x, max_y):
#     spring = (500 - min_x, 0)
#     grid[spring[1]][spring[0]] = '+'
#     queue = deque([spring])
    
#     while queue:
#         x, y = queue.popleft()
#         if y > max_y:
#             continue

#         # Flow downward
#         if y + 1 <= max_y and grid[y + 1][x] == '.':
#             grid[y + 1][x] = '|'
#             queue.append((x, y + 1))
#             continue

#         # Check if downward flow is blocked
#         if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
#             left, right = x, x
#             left_blocked, right_blocked = False, False
            
#             # Spread left
#             while grid[y][left - 1] == '.':
#                 left -= 1
#                 grid[y][left] = '|'
#                 if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
#                     queue.append((left, y))
#                     break
#             else:
#                 left_blocked = grid[y][left - 1] in ('#', '~')

#             # Spread right
#             while grid[y][right + 1] == '.':
#                 right += 1
#                 grid[y][right] = '|'
#                 if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
#                     queue.append((right, y))
#                     break
#             else:
#                 right_blocked = grid[y][right + 1] in ('#', '~')

#             # If the row is enclosed, convert to settled water
#             if left_blocked and right_blocked:
#                 for i in range(left, right + 1):
#                     grid[y][i] = '~'
#                 queue.append((x, y - 1))  # Recheck row above for settling
def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Spread left and right if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            left_blocked, right_blocked = False, False
            
            # Spread left
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
                    queue.append((left, y))
                    break
            else:
                left_blocked = grid[y][left - 1] in ('#', '~')

            # Spread right
            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
                    queue.append((right, y))
                    break
            else:
                right_blocked = grid[y][right + 1] in ('#', '~')

            # Handle joined falling streams
            if not left_blocked and not right_blocked:
                # If both ends are unblocked, let the water flow freely
                queue.append((left, y))
                queue.append((right, y))
                continue

            # If the row is enclosed, convert to settled water
            if left_blocked and right_blocked:
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling


def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))
def process_paired_rows(grid):
    """
    Processes rows with the pattern:
    #, followed by |, ending with #, where the row below is fully supportive.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 1):  # Stop at the second-to-last row
        row = grid[y]
        below_row = grid[y + 1]
        
        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a valid segment of | bounded by #
                while x < width and row[x] == '|':
                    x += 1
                
                if x < width and row[x] == '#':  # End boundary found
                    end = x
                    
                    # Verify the below row has matching boundaries
                    if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] == '#' for i in range(start + 1, end)):
                        # Replace | with ~ in the current row
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1
def save_grid_to_file(grid, file_name='output-001.txt'):
    """
    Saves the grid to a file instead of displaying it.
    """
    with open(file_name, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')

# Main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    process_paired_rows(grid)
    save_grid_to_file(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
if __name__ == "__main__":
    main('input.txt')


Total water tiles (~ or |): 930


In [18]:
def process_grid_file(input_file, output_file):
    """
    Reads a grid from an input file, applies process_paired_rows, 
    and writes the resulting grid to an output file.
    """
    with open(input_file, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    
    # Apply the paired rows processing
    process_paired_rows(grid)
    
    # Save the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')

# Example usage
if __name__ == "__main__":
    input_file = 'output-001.txt'
    output_file = 'output-processed-grid.txt'
    process_grid_file(input_file, output_file)
    print(f"Processed grid saved to {output_file}")


Processed grid saved to output-processed-grid.txt


In [26]:
def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()
# def process_supported_rows(grid):
#     """
#     Converts rows containing '|' (and optionally '.') bounded by '#' into '~',
#     provided the row below fully supports the transformation with only '#' and '~'.
#     """
#     height = len(grid)
#     width = len(grid[0])
    
#     for y in range(height - 1):  # Stop at the second-to-last row
#         row = grid[y]
#         below_row = grid[y + 1]
        
#         x = 0
#         while x < width:
#             # Look for a starting boundary
#             if row[x] == '#':
#                 start = x
#                 x += 1
                
#                 # Check for a segment of valid characters ('|' and optionally '.')
#                 while x < width and row[x] in {'|', '.'}:
#                     x += 1
                
#                 if x < width and row[x] == '#':  # End boundary found
#                     end = x
                    
#                     # Check if the row below has matching boundaries and is supportive
#                     if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] in {'#', '~'} for i in range(start + 1, end)):
#                         # Replace all | and . in the current row with ~
#                         for i in range(start + 1, end):
#                             row[i] = '~'
#             else:
#                 x += 1
def process_supported_rows(grid):
    """
    Converts rows containing '|' (and optionally '.') bounded by '#' into '~',
    provided the row below fully supports the transformation with only '#' and '~'.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 1):  # Stop at the second-to-last row
        row = grid[y]
        below_row = grid[y + 1]
        
        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a segment of valid characters ('|' and '.')
                has_pipe = False
                while x < width and row[x] in {'|', '.'}:
                    if row[x] == '|':
                        has_pipe = True
                    x += 1
                
                if x < width and row[x] == '#' and has_pipe:  # End boundary found and at least one '|'
                    end = x
                    
                    # # Validate the supporting row below
                    # if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] in {'#', '~'} for i in range(start + 1, end)):
                    #     # Replace all | and . in the current row with ~
                    #     for i in range(start + 1, end):
                    #         row[i] = '~'
                                # Validate the supporting row below
                    if all(below_row[i] in {'#', '~'} for i in range(start + 1, end)):
                        # Replace all | and . in the current row with ~
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1

def process_grid_with_supported_rows(input_file, output_file):
    """
    Reads a grid from an input file, applies process_supported_rows, 
    and writes the resulting grid to an output file.
    """
    with open(input_file, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    
    # Apply the supported rows processing
    process_supported_rows(grid)
    
    # Save the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')

# Example usage
if __name__ == "__main__":
    test_input_file = 'output-001.txt'
    test_output_file = 'output-series-001-member-001.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")


Processed grid saved to output-series-001-member-001.txt


In [27]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-001.txt'
    test_output_file = 'output-series-001-member-002.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-002.txt


In [28]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-002.txt'
    test_output_file = 'output-series-001-member-003.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-003.txt


In [32]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-003.txt'
    test_output_file = 'output-series-001-member-004.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-004.txt


In [33]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-004.txt'
    test_output_file = 'output-series-001-member-005.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-005.txt


In [34]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-005.txt'
    test_output_file = 'output-series-001-member-006.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-006.txt


In [35]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-006.txt'
    test_output_file = 'output-series-001-member-007.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-007.txt


In [36]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-007.txt'
    test_output_file = 'output-series-001-member-008.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-008.txt


In [37]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-008.txt'
    test_output_file = 'output-series-001-member-009.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-009.txt


In [38]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-009.txt'
    test_output_file = 'output-series-001-member-010.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-010.txt


In [39]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-010.txt'
    test_output_file = 'output-series-001-member-011.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-011.txt


In [40]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-series-001-member-011.txt'
    test_output_file = 'output-series-001-member-012.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-series-001-member-012.txt


In [23]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-processed-grid-001.txt'
    test_output_file = 'output-processed-grid-002.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-processed-grid-002.txt


In [44]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Spread left and right if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, add to queue
                    queue.append((left, y))
                    break

            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, add to queue
                    queue.append((right, y))
                    break

            # Add left and right edges to queue for further processing
            if grid[y][left - 1] in ('#', '~') and grid[y][right + 1] in ('#', '~'):
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling

    # Post-process: Convert bounded '|' to '~'
    while True:
        changes = False
        for y in range(max_y + 1):
            row = ''.join(grid[y])
            # Replace bounded '|' sequences with '~'
            new_row = re.sub(r'#\|+\#', lambda m: '#' + '~' * (len(m.group()) - 2) + '#', row)
            if new_row != row:
                grid[y] = list(new_row)
                changes = True

        if not changes:
            break





def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))

# Main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    # display_grid(grid)
    process_grid_to_file(grid, test_output_file)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
if __name__ == "__main__":
    main('input.txt')


Total water tiles (~ or |): 1096


In [47]:
def process_capped_rows(grid):
    """
    Processes rows where:
    - A row contains '|' above a row with '#' at the ends and '~' in the middle.
    - Transforms the row with '|' into a fully bordered row of '|', 
      with '|#' on each side of the supporting row.

    Args:
        grid (list of list of str): The grid to process.

    Modifies:
        The grid in place.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(1, height):  # Start from the second row
        current_row = grid[y]
        above_row = grid[y - 1]
        
        # Check if the current row has the required pattern
        if (
            current_row[0] == '#' and current_row[-1] == '#' and
            all(cell in {'#', '~'} for cell in current_row[1:-1])
        ):
            # Transform the row above
            for x in range(width):
                if above_row[x] == '.' or above_row[x] == '|':  # Valid characters to replace
                    above_row[x] = '|'
                else:
                    above_row[x] = above_row[x]  # Preserve boundaries if already valid

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

def process_grid_with_capped_rows(input_file, output_file):
    """
    Reads a grid from an input file, applies process_capped_rows,
    and writes the resulting grid to an output file.
    """
    with open(input_file, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    
    # Apply the capped rows processing
    process_capped_rows(grid)
    
    # Save the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')

# Example usage
if __name__ == "__main__":
    test_input_file = 'output-processed-grid.txt'
    test_output_file = 'output-series-002-member-001.txt'
    process_grid_with_capped_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")


Processed grid saved to output-series-002-member-001.txt


In [81]:
def process_grid_and_save(grid, output_file):
    """
    Processes a grid using process_capped_rows and process_vertical_pipes,
    then writes the final grid to an output file.

    Args:
        grid (list of list of str): The grid to process.
        output_file (str): The file to write the processed grid to.
    """
    # Apply process_capped_rows twice
    process_capped_rows(grid)
    process_capped_rows(grid)
    
    # Apply process_vertical_pipes
    process_vertical_pipes(grid)
    
    # Save the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')
def read_grid_from_file(input_file):
    """
    Reads a grid from a file.

    Args:
        input_file (str): The file to read.

    Returns:
        list of list of str: The grid as a list of rows.
    """
    with open(input_file, 'r') as file:
        return [list(line.strip()) for line in file.readlines()]

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

# Example usage
if __name__ == "__main__":
    input_file = 'output-processed-grid.txt'
    output_file = 'output-series-003-member-001.txt'
    
    # Read the grid from the input file
    grid = read_grid_from_file(input_file)
    
    # Display the grid before processing
    print("Before Processing:")
    display_grid(grid)
    
    # Process the grid and save to the output file
    process_grid_and_save(grid, output_file)
    
    # Confirm the output
    #print(f"Processed grid saved to {output_file}")


Before Processing:
......................................................................................................................................................................................................................................................................................................................+........................................
......................................................................................................................................................................................................................................................................................................................|........................................
.....................................................................................................................................................................................................................................................................................

In [48]:
def test_process_capped_rows():
    grid = [
        list("...........|.............."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#.")
    ]
    print("Before Processing:")
    display_grid(grid)
    
    process_capped_rows(grid)
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows()


Before Processing:
...........|..............
.#~~~~~~~~~~~~~~~~~~~~~~#.

After Processing:
...........|..............
.#~~~~~~~~~~~~~~~~~~~~~~#.



In [82]:
def test_combined_processing():
    """
    Tests the combination of process_capped_rows and process_vertical_pipes
    on the given input grid.
    """
    # Input grid
    grid = [
        list("...........|.............."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#."),
        list(".#~~~~######~~~~~~~~~~~~#."),
        list(".#~~~~#....#~~~~~~~~~~~~#."),
        list(".#~~~~#....#~~~~~~~~~~~~#."),
        list(".#~~~~#....#~~~~~~~~~~~~#."),
        list(".#~~~~######~~~~~~~~~~~~#."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#."),
        list(".########################."),
        list(".........................."),
    ]
    
    print("Before Processing:")
    display_grid(grid)
    
    # Process the grid
    process_capped_rows(grid)  # First pass of capped rows
    process_capped_rows(grid)  # Second pass of capped rows
    process_vertical_pipes(grid)  # Apply vertical pipes processing
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_combined_processing()


Before Processing:
...........|..............
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~######~~~~~~~~~~~~#.
.#~~~~#....#~~~~~~~~~~~~#.
.#~~~~#....#~~~~~~~~~~~~#.
.#~~~~#....#~~~~~~~~~~~~#.
.#~~~~######~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.########################.
..........................

I found .#~~~~~~~~~~~~~~~~~~~~~~#. with ...........|.............. on top.
I found .#~~~~~~~~~~~~~~~~~~~~~~#. with .||||||||||||||||||||||||. on top.
After Processing:
||||||||||||||||||||||||||
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~######~~~~~~~~~~~~#.
.#~~~~#....#~~~~~~~~~~~~#.
.#~~~~#....#~~~~~~~~~~~~#.
.#~~~~#....#~~~~~~~~~~~~#.
.#~~~~######~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.#~~~~~~~~~~~~~~~~~~~~~~#.
.########################.
..........................



In [96]:
def process_file_and_save(input_file, output_file):
    """
    Reads a grid from the input file, applies process_capped_rows and process_vertical_pipes,
    and writes the final grid to the output file.

    Args:
        input_file (str): The path to the input file.
        output_file (str): The path to the output file.
    """
    # Read the grid from the input file
    with open(input_file, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    
    # Apply the processing steps
    process_capped_rows(grid)  # First pass of capped rows
    # process_capped_rows(grid)  # Second pass of capped rows
    # process_vertical_pipes(grid)  # Apply vertical pipes processing
    
    # Write the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')
if __name__ == "__main__":
    input_file = 'focused-test-001.txt'
    output_file = 'output-series-006-member-001.txt'
    
    # Process the file and save the result
    process_file_and_save(input_file, output_file)
    
    print(f"Processed grid saved to {output_file}")


Checking row 1: ..........................................|............................
Row above: ..........................................+............................
Row 1 ignored: fewer than two '#' characters.
Checking row 2: ..........................................|............................
Row above: ..........................................|............................
Row 2 ignored: fewer than two '#' characters.
Checking row 3: ..........................................|............................
Row above: ..........................................|............................
Row 3 ignored: fewer than two '#' characters.
Checking row 4: ..........................................|............................
Row above: ..........................................|............................
Row 4 ignored: fewer than two '#' characters.
Checking row 5: ..........#...............................|............................
Row above: .................................

In [91]:
def process_file_and_save(input_file, output_file):
    """
    Reads a grid from the input file, applies process_capped_rows and process_vertical_pipes,
    and writes the final grid to the output file.

    Args:
        input_file (str): The path to the input file.
        output_file (str): The path to the output file.
    """
    # Read the grid from the input file
    with open(input_file, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    
    # Apply the processing steps
    process_capped_rows(grid)  # First pass of capped rows
    # process_capped_rows(grid)  # Second pass of capped rows
    # process_vertical_pipes(grid)  # Apply vertical pipes processing
    
    # Write the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')
if __name__ == "__main__":
    input_file = 'focused-test-001.txt'
    output_file = 'output-series-006-member-001.txt'
    
    # Process the file and save the result
    process_file_and_save(input_file, output_file)
    
    print(f"Processed grid saved to {output_file}")

Checking row 1: ..........................................|............................
Row above: ..........................................+............................
Row 1 ignored: fewer than two '#' characters.
Checking row 2: ..........................................|............................
Row above: ..........................................|............................
Row 2 ignored: fewer than two '#' characters.
Checking row 3: ..........................................|............................
Row above: ..........................................|............................
Row 3 ignored: fewer than two '#' characters.
Checking row 4: ..........................................|............................
Row above: ..........................................|............................
Row 4 ignored: fewer than two '#' characters.
Checking row 5: ..........#...............................|............................
Row above: .................................

In [95]:
def test_process_capped_rows_string():
    """
    Tests a single pass of process_capped_rows on the given input grid as a string.
    """
    # Input grid as a string
    input_grid = """
..........|..............
..........|..............
...#|||||||||||||||||||..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...###################|..
......................|..
"""
    # Convert the string input to a grid (list of lists)
    grid = [list(line) for line in input_grid.strip().split('\n')]
    
    print("Before Processing:")
    display_grid(grid)
    
    # Apply one pass of process_capped_rows
    process_capped_rows(grid)
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows_string()


Before Processing:
..........|..............
..........|..............
...#|||||||||||||||||||..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...#~~~~~~~~~~~~~~~~~#|..
...###################|..
......................|..

Checking row 1: ..........|..............
Row above: ..........|..............
Row 1 ignored: fewer than two '#' characters.
Checking row 2: ...#|||||||||||||||||||..
Row above: ..........|..............
Row 2 ignored: fewer than two '#' characters.
Checking row 3: ...#~~~~~~~~~~~~~~~~~#|..
Row above: ...#|||||||||||||||||||..
Row 3 ignored: not enclosed by dots.
Checking row 4: ...#~~~~~~~~~~~~~~~~~#|..
Row above: ...#~~~~~~~~~~~~~~~~~#|..
Row 4 ignored: not enclosed by dots.
Checking row 5: ...#~~~~~~~~~~~~~~~~~#|..
Row above: ...

In [97]:
def test_process_capped_rows_string():
    """
    Tests a single pass of process_capped_rows on the given input grid as a string.
    """
    # Input grid as a string
    input_grid = """
...........|...............
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~######~~~~~~~~~~~~#..
.#~~~~#....#~~~~~~~~~~~~#..
.#~~~~#....#~~~~~~~~~~~~#..
.#~~~~#....#~~~~~~~~~~~~#..
.#~~~~######~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.########################..
...........................

"""
    # Convert the string input to a grid (list of lists)
    grid = [list(line) for line in input_grid.strip().split('\n')]
    
    print("Before Processing:")
    display_grid(grid)
    
    # Apply one pass of process_capped_rows
    process_capped_rows(grid)
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows_string()


Before Processing:
...........|...............
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~######~~~~~~~~~~~~#..
.#~~~~#....#~~~~~~~~~~~~#..
.#~~~~#....#~~~~~~~~~~~~#..
.#~~~~#....#~~~~~~~~~~~~#..
.#~~~~######~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.#~~~~~~~~~~~~~~~~~~~~~~#..
.########################..
...........................

Checking row 1: .#~~~~~~~~~~~~~~~~~~~~~~#..
Row above: ...........|...............
Valid match: Supporting row .#~~~~~~~~~~~~~~~~~~~~~~#.. with row above ...........|................
Checking row 2: .#~~~~~~~~~~~~~~~~~~~~~~#..
Row above: .#~~~~~~~~~~~~~~~~~~~~~~#..
Row 2 ignored: no '|' found in the row above within bounds.
Checking row 3: .#~~~~~~~~~~~~~~~~~~~~~~#..
Row above: .#~~~~~~~~~~~~~~~~~~~~~~#..
Row 3 ignored: no '|' found in the row above within bounds.
Checking row 4: .#~~~~######~~~~~~~~~~~~#..
Row above: .#~~~~~~~~~~~~~~~~~~~~~~#..
Row 4 ignored: no '|' found in the r

In [101]:
def process_capped_rows(grid):
    """
    Processes rows where:
    - A row starts and ends with '#', and the middle contains only '#' or '~'.
    - The row above contains `.` or `|`, bounded by `#`.

    If valid, replaces all `.` and `|` in the above row within the bounds with `|`.

    Args:
        grid (list of list of str): The grid to process.

    Modifies:
        The grid in place.
    """
    height = len(grid)
    width = len(grid[0])

    for y in range(1, height):  # Start from the second row
        current_row = grid[y]
        above_row = grid[y - 1]

        x = 0
        while x < width:
            # Find the next segment starting with '#'
            if current_row[x] == '#':
                start = x
                # Find the end of this segment
                while x < width and current_row[x] in {'#', '~'}:
                    x += 1
                end = x - 1  # The last valid index

                # Check if the segment is valid
                if end > start:
                    # Ensure the row above contains only `.` or `|` within the bounds
                    if all(cell in {'.', '|'} for cell in above_row[start:end + 1]):
                        # Debug statement for identified valid match
                        print(
                            f"Valid match: Supporting row {''.join(current_row)} "
                            f"with row above {''.join(above_row)} in range [{start}, {end}]"
                        )

                        # Replace all `.` and `|` in the above row within the bounds with `|`
                        for i in range(start, end + 1):
                            above_row[i] = '|'
            x += 1

def test_process_capped_rows_string():
    """
    Tests a single pass of process_capped_rows on the given input grid as a string.
    """
    input_grid = """
...............#...............#..................................###############...................................#...............#......################..........#....................................................................................................#.......................#................................|...........................
...............#...............#..............................................................#.....................#...............#.........................#......#....................................................................................................#.......................#................................|...........................
...............#...............#..............................................................#............#........#...............#.........................#......#....................................................................................................#.......................#................................|...........................
...............#...............#......................................#....#..................#............#........#...............#.........................#......#........#...........................................................................................#.......................#......................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................................#....#..................#............#........#...............#.........................#......#........#........................#..................................................................#.......................#......................#~~~~~~~~~~~~~~~~~~~~~~#..............
"""
    # Convert the string input to a grid (list of lists)
    grid = [list(line) for line in input_grid.strip().split('\n')]

    print("Before Processing:")
    display_grid(grid)

    # Apply one pass of process_capped_rows
    process_capped_rows(grid)

    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows_string()


Before Processing:
...............#...............#..................................###############...................................#...............#......################..........#....................................................................................................#.......................#................................|...........................
...............#...............#..............................................................#.....................#...............#.........................#......#....................................................................................................#.......................#................................|...........................
...............#...............#..............................................................#............#........#...............#.........................#......#....................................................................................................#..........

In [102]:
def test_process_capped_rows_string():
    """
    Tests a single pass of process_capped_rows on the given input grid as a string.
    """
    # Input grid as a string
    input_grid = """
......................................................................................................................................................................................................................................................................................................................+........................................
......................................................................................................................................................................................................................................................................................................................|........................................
......................................................................................................................................................................................................................................................................................................................|........................................
......................................................................................................................................................................................................................................................................................................................|........................................
......................................................................................................................................................................................................................................................................................................................|........................................
......................................................................................................................................................................................................................................................................................#...............................|........................................
......................................................................................................................................................................................................................................................................................#.............#.................|........................................
......................................................................................................................................................................................................................................................................................#.............#.................|........................................
......................................................................................................................................................................................................................................................................................#.............#.................|........................................
......................................................................................................................................................#...............................................................................................................................#.............#.................|........................................
......................................................................................................................................................#...........................#...................................................................................................#.............#.................|........................................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....||||||||||||||||||||||#..............................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
......................................................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#.....................................................................................................................#...........................#............#......................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#...#.................................................................................................................#...........................#........#...#......................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#...#.................................................................................................................#...........................#........#...#......................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#...#.................................................................................................................#...........................#........#...#......................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#...#.................................................................................................................#...........................#........#####......................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#...#.................................................................................................................#...........................#...................................................................................................#.............#.....|#~~~~~~~~~~~~~~~~~~~~#..............................
................................#...#.................................................................................................................#############################...................................................................................................#.............#.....|######################..............................
................................#...#.................................................................................................................................................................................................................................................#.............#.....|....................................................
................................#...#.................................................................................................................................................................................................................................................#.............#.....|....................................................
................................#...#.................................................................................................................................................................................................................................................###############.....|....................................................
................................#...#.....................................................................................................................................................................................................................................................................|....................................................
................................#...#..........................................................#..........................................................................................................................................................................................................|....................................................
................................#...#..........................................#...............#..........................................................................................................................................................................................................|....................................................
................................#...#..........................................#...............#...................................................................................................................................................................................................#|||||||||||||||||||........................................
................................#...#..........................................#...............#...................................................................................................................................................................................................#~~~~~~~~~~~~~~~~~#|........................................
................................#...#..........................................#...............#...................................................................................................................................................................................................#~~~~~~~~~~~~~~~~~#|........................................
................................#...#..........................................#...............#..............................................................................................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
................................#...#..........................................#...............#..............................................................................................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
................................#####..........................................#......#.#......#..............................................................................................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#......#.#......#..............................................................................................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#......#.#......#..............................................................................................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#......#.#......#............................................#.................................................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#......#.#......#............................................#........................#........................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#......###......#............................................#........................#........................................#......#.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#...............#............................................#........................#........................................########.............................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#...............#............................................#........................#.............................................................................................................................#~~~~~~~~~~~~~~~~~#|........................................
...............................................................................#################............................................#.......#.....#..........#.............................................................................................................................###################|........................................
............................................................................................................................................#.......#.....#..........#................................................................................................................................................|........................................
............................................................................................................................................#.......#.....#..........#................................................................................................................................................|........................................
............................................................................................................................................#.......#.....#..........#................................................................................................................................................|........................................
............................................................................................................................................#.......#######..........#................................................................................................................................................|........................................
.................................................................#..........................................................................#........................#................................................................................................................................................|........................................
.................................................................#.....#....................................................................#........................#................................................................................................................................................|........................................
.................................................................#.....#....................................................................#........................#.....................................................................................#..........................................................|........................................
.#...............................................................#.....#....................................................................#........................#.....................................................................................#........#.................................................|........................................
.#..............#................................................#.....#....................................................................#........................#.....................................................................................#........#.................................................|........................................
.#..............#................................................#.....#....................................................................#........................#.....................................................................................#........#.................................................|........................................
.#..............#................................................#.....#.....................................#.................#............#........................#.....................................................................................#........#........................................#||||||||||||||||||||.............................
.#..............#................................................#.....#.....................................#.................#............#........................#.....................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#............#........................#.....................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#............#........................#.....................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#............#........................#.....................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#............##########################.....................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#...........................................................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#...........................................................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#...........................................................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#...........................................................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#...........................................................................................................................#........#........................................#~~~~~~~~~~~~~~~~~~#|.............................
.#..............#................................................#.....#.....................................#.................#.........................................................................................................#.................#........#........................................####################|.............................
.#..............#................................................#.....#.....................................#.................#..................................................................#................#.....................#.................#........#............................................................|.............................
.#..............#................................................#.....#.....................................#.................#.............................................#....................#................#.....................#.................#........#............................................................|.............................
.#..............#................................................#.....#.....................................#.................#.............................................#....................#................#.....................#.................#........#............................................................|.............................
.#..............#................................................#.....#.....................................#.................#.............................................#....................#................#.....................#.................#........#............................................................|.............................
.#..............#................................................#.....#.....................................#.................#.............................................#....................#................#.....................#.................##########.................................................#..#.......|.............................
.#..............#................................................#.....#.....................................#.................#.............................................#....................#................#.....................#............................................................................#..#.......|.............................
.#..............#................................................#.....#.....................................#......####.......#.............................................#....................#................#.....................#.....................................................................#......#..#||||||||||...........................
.#..............#................................................#.....#.....................................#......#..#.......#.............................................#....................#................#.....................#.....................................................................#......#..#~~~~~~~~#|...........................
.#..............#................................................#.....#.....................................#......#..#.......#.............................................#.........#####......#................#.........#....#......#.....................................................................#......#..#~~~~~~~~#|...........................
.#..............#................................................#######.....................................#......#..#.......#.............................................#.........#...#......#................#.........#....#......#.....................................................................#......#..#~~~~~~~~#|...........................
.################............................................................................................#......#..#.......#.............................................#.........#####......#................#.........#....#......#.....................................................................#......#..#~~~~~~~~#|...........................
.............................................................................................................#......####.......#..........................#..................#....................#................#.........#....#......#.....................................................................#......#..#~~~~~~~~#|...........................
.............................................................................................................#.................#...........#..............#..................#....................#................#.........#....#......#.....................................................................#......#..#~~~~~~~~#|...........................
.............................................................................................................#.................#...........#..............#..................#....................#................#.........#....#......#.....................................................................#......#..#~~~~~~~~#|...........................
.............................................................................................................#.................#...........#..............#..................######################................#.........#....#......#................................#....................................#......####~~~~~~~~#|...........................
...............#...............#..................................#..........................................#.................#...........#..............#........................................................#.........######......#................................#.......................#............#~~~~~~~~~~~~~~~~~~#|...........................
..#.......#....#...............#..................................#.............#............................###################...........#..............#........................................................#.....................#................................#.......................#............#~~~~~~~~~~~~~~~~~~#|...........................
..#.......#....#...............#..................................#.............#..........................................................#..............#........................................................#.....................#................................#.......................#............####################|...........................
..#.......#....#...............#..................................#.............#..........................................................#..............#........................................................#.....................#................................#.......................#................................|...........................
..#.......#....#...............#..................................#.............#..........................................................#..............#........................................................#.....................#................................#.......................#................................|...........................
..#.......#....#...............#..................................#.............#..........................................................#..............#........................................................#######################................................#.......................#................................|...........................
..#.......#....#...............#..................................#.............#..........................................................#..............#...............................................................................................................#.......................#................................|...........................
..#########....#...............#..................................#.............#..........................................................#..............#...............................................................................................................#.......................#................................|...........................
...............#...............#..................................#.............#..........................................................#..............#...............................................................................................................#.......................#................................|...........................
...............#...............#..................................###############...................................#...............#......################..........#....................................................................................................#.......................#................................|...........................
...............#...............#..............................................................#.....................#...............#.........................#......#....................................................................................................#.......................#................................|...........................
...............#...............#..............................................................#............#........#...............#.........................#......#....................................................................................................#.......................#................................|...........................
...............#...............#......................................#....#..................#............#........#...............#.........................#......#........#...........................................................................................#.......................#......................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................................#....#..................#............#........#...............#.........................#......#........#........................#..................................................................#.......................#......................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................................#....#..................#............#........#...............#.........................#......#........#........................#..................................................................#.......................#......................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................................#....#..................##############........#...............#.........................########........#........................#.............................................................#....#.......................#......................#~~~~######~~~~~~~~~~~~#..............
...............#...............#......................................#....#........................................#...............#.........................................#......####..............#.......................................................#.....#....#.......................#......................#~~~~#....#~~~~~~~~~~~~#..............
...............#...............#......................................#....#........................................#...............#.........................................#......#..#..............#.......................................................#.....#....#.......................#......................#~~~~#....#~~~~~~~~~~~~#..............
...............#....######.....#......................................#....#........................................#...............#.........................................#......#..#..............#.......................................................#.....#....#.......................#......................#~~~~#....#~~~~~~~~~~~~#..............
...............#....#....#.....#......................#..........#....#....#........................................#################.........................................#......#..#..............#.......................................................#.....#....#.......................#......................#~~~~######~~~~~~~~~~~~#..............
...............#....######.....#......................#..........#....######..................................................................................................#......#..#..............#.......................................................#.....#....#.......................#......................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................#..........#............................................................................................................#......#..#..............#.....................#.................................#.....#....#########################......................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................#..........#............................................................................................................#......#..#..............#.......#.............#.................................#.....#...................................................#~~~~~~~~~~~~~~~~~~~~~~#..............
...............#...............#......................#..........#............................................................................................................#......#..#..............#.......#.............#.................................#.....#...................................................########################..............
...............#...............#......................#..........#......................................................................#.....................................#......#..#..............#.......#.............#.................................#.....#.........................................................................................
"""
    # Convert the string input to a grid (list of lists)
    grid = [list(line) for line in input_grid.strip().split('\n')]
    
    print("Before Processing:")
    display_grid(grid)
    
    # Apply one pass of process_capped_rows
    process_capped_rows(grid)
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows_string()


Before Processing:
......................................................................................................................................................................................................................................................................................................................+........................................
......................................................................................................................................................................................................................................................................................................................|........................................
.....................................................................................................................................................................................................................................................................................

In [94]:
def process_capped_rows(grid):
    """
    Processes rows where:
    - A row starts and ends with '.', followed by '#' and ending with '#' (e.g., .#...#.).
    - The middle contains only '#' or '~'.
    - The row above contains at least one '|', strictly within the bounds of the row below.

    Args:
        grid (list of list of str): The grid to process.

    Modifies:
        The grid in place.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(1, height):  # Start from the second row
        current_row = grid[y]
        above_row = grid[y - 1]
        
        # Debugging: Print the rows being checked
        print(f"Checking row {y}: {''.join(current_row)}")
        print(f"Row above: {''.join(above_row)}")
        
        # Validate the current row
        if current_row.count('#') < 2:
            print(f"Row {y} ignored: fewer than two '#' characters.")
            continue  # Ignore rows without at least two `#`
        
        # Find the bounds of the `#` section
        start = current_row.index('#')
        end = len(current_row) - 1 - current_row[::-1].index('#')
        
        # Ensure the middle section of the current row is valid (`#` or `~` only)
        if not all(cell in {'#', '~'} for cell in current_row[start:end + 1]):
            print(f"Row {y} ignored: invalid characters in the middle section.")
            continue
        
        # Ensure the row starts and ends with a dot before the `#` boundaries
        if not (start > 0 and current_row[start - 1] == '.' and end < width - 1 and current_row[end + 1] == '.'):
            print(f"Row {y} ignored: not enclosed by dots.")
            continue
        
        # Check the row above contains at least one `|` strictly within the bounds
        if not any(cell == '|' for cell in above_row[start:end + 1]):
            print(f"Row {y} ignored: no '|' found in the row above within bounds.")
            continue
        
        # Debug statement: Valid match
        print(f"Valid match: Supporting row {''.join(current_row)} with row above {''.join(above_row)}.")
        
        # Replace the portion of the above row within the bounds with `|`
        for x in range(start, end + 1):
            above_row[x] = '|'


In [75]:
def test_process_capped_rows():
    grid = [
        list(".............|.........."),
        list("#~~~~~~~~~~~~~~~~~~~~~~#")
    ]
    print("Before Processing:")
    display_grid(grid)
    
    process_capped_rows(grid)
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows()


Before Processing:
.............|..........
#~~~~~~~~~~~~~~~~~~~~~~#

I found #~~~~~~~~~~~~~~~~~~~~~~# with .............|.......... on top.
After Processing:
||||||||||||||||||||||||
#~~~~~~~~~~~~~~~~~~~~~~#



In [78]:
def test_process_capped_rows():
    grid = [
        list("..............|..........."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#.")
    ]
    print("Before Processing:")
    display_grid(grid)
    
    process_capped_rows(grid)
    
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows()

Before Processing:
..............|...........
.#~~~~~~~~~~~~~~~~~~~~~~#.

I found .#~~~~~~~~~~~~~~~~~~~~~~#. with ..............|........... on top.
After Processing:
.||||||||||||||||||||||||.
.#~~~~~~~~~~~~~~~~~~~~~~#.



In [77]:
def process_vertical_pipes(grid):
    """
    Processes vertical pipes (`|`) in the grid:
    - Replaces a `.|` followed by a `.#` with `||` followed by `.#`.
    - Replaces a `|.` followed by a `#.` with `||` followed by `#.`.

    Args:
        grid (list of list of str): The grid to process.

    Modifies:
        The grid in place.
    """
    height = len(grid)
    width = len(grid[0])

    for y in range(1, height):  # Start from the second row
        above_row = grid[y - 1]
        current_row = grid[y]
        
        for x in range(width - 1):  # Ensure we don't go out of bounds
            # Check for `.|` followed by `.#`
            if above_row[x] == '.' and above_row[x + 1] == '|' and current_row[x] == '.' and current_row[x + 1] == '#':
                above_row[x] = '|'
            
            # Check for `|.` followed by `#.`
            if above_row[x] == '|' and above_row[x + 1] == '.' and current_row[x] == '#' and current_row[x + 1] == '.':
                above_row[x + 1] = '|'


In [79]:
def test_process_capped_rows():
    grid = [
        list("..............|..........."),
        list(".#~~~~~~~~~~~~~~~~~~~~~~#.")
    ]
    print("Before Processing:")
    display_grid(grid)
    
    process_capped_rows(grid)
    process_vertical_pipes(grid)
    print("After Processing:")
    display_grid(grid)

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

if __name__ == "__main__":
    test_process_capped_rows()

Before Processing:
..............|...........
.#~~~~~~~~~~~~~~~~~~~~~~#.

I found .#~~~~~~~~~~~~~~~~~~~~~~#. with ..............|........... on top.
After Processing:
||||||||||||||||||||||||||
.#~~~~~~~~~~~~~~~~~~~~~~#.



In [42]:
import re

def count_water_and_flowing(grid):
    """
    Counts the occurrences of '~' and '|' in the grid.
    """
    return sum(row.count('~') + row.count('|') for row in grid)

def process_until_stable(grid):
    """
    Repeatedly applies process_supported_rows until the number of '~' and '|'
    occurrences does not change.
    """
    previous_count = -1
    current_count = count_water_and_flowing(grid)
    
    while current_count != previous_count:
        previous_count = current_count
        process_supported_rows(grid)
        current_count = count_water_and_flowing(grid)

def process_grid_with_supported_rows(input_file, output_file):
    """
    Reads a grid from an input file, applies process_until_stable,
    and writes the resulting grid to an output file.
    """
    with open(input_file, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    
    # Apply the supported rows processing repeatedly until stable
    process_until_stable(grid)
    
    # Save the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')

# Example usage
if __name__ == "__main__":
    test_input_file = 'input-grid.txt'
    test_output_file = 'output-processed-grid.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")


FileNotFoundError: [Errno 2] No such file or directory: 'input-grid.txt'

In [43]:
def process_grid_to_file(grid, output_file):
    """
    Processes the input grid until stable and writes the result to an output file.

    Args:
        grid (list of list of str): The grid to process.
        output_file (str): The path to the output file.
    """
    # Apply the supported rows processing repeatedly until stable
    process_until_stable(grid)
    
    # Save the processed grid to the output file
    with open(output_file, 'w') as file:
        for row in grid:
            file.write(''.join(row) + '\n')


In [24]:
# Example usage
if __name__ == "__main__":
    test_input_file = 'output-processed-grid-002.txt'
    test_output_file = 'output-processed-grid-003.txt'
    process_grid_with_supported_rows(test_input_file, test_output_file)
    print(f"Processed grid saved to {test_output_file}")

Processed grid saved to output-processed-grid-003.txt


In [31]:
def process_supported_rows(grid):
    """
    Converts rows containing '|' (and optionally '.') bounded by '#' into '~',
    provided the row below fully supports the transformation with only '#' and '~'.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 1):  # Stop at the second-to-last row
        row = grid[y]
        below_row = grid[y + 1]
        
        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a segment of valid characters ('|' and '.')
                has_pipe = False
                while x < width and row[x] in {'|', '.'}:
                    if row[x] == '|':
                        has_pipe = True
                    x += 1
                
                if x < width and row[x] == '#' and has_pipe:  # End boundary found and at least one '|'
                    end = x
                    
                    # # Validate the supporting row below
                    # if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] in {'#', '~'} for i in range(start + 1, end)):
                    #     # Replace all | and . in the current row with ~
                    #     for i in range(start + 1, end):
                    #         row[i] = '~'
                                # Validate the supporting row below
                    if all(below_row[i] in {'#', '~'} for i in range(start + 1, end)):
                        # Replace all | and . in the current row with ~
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1

In [14]:
def process_paired_rows(grid):
    """
    Processes rows with the pattern:
    #, followed by |, ending with #, where the row below is fully supportive.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 1):  # Stop at the second-to-last row
        row = grid[y]
        below_row = grid[y + 1]
        
        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a valid segment of | bounded by #
                while x < width and row[x] == '|':
                    x += 1
                
                if x < width and row[x] == '#':  # End boundary found
                    end = x
                    
                    # Verify the below row has matching boundaries
                    if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] == '#' for i in range(start + 1, end)):
                        # Replace | with ~ in the current row
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

# Example usage with the main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    
    # Apply the pattern processing function
    process_paired_rows(grid)
    
    display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Test Example
if __name__ == "__main__":
    test_grid = [
        list(".........#||||||||||||||||||||||#......................"),
        list(".................########################..............")
    ]
    print("Before Processing:")
    display_grid(test_grid)
    process_paired_rows(test_grid)
    print("After Processing:")
    display_grid(test_grid)


Before Processing:
.........#||||||||||||||||||||||#......................
.................########################..............

After Processing:
.........#||||||||||||||||||||||#......................
.................########################..............



In [15]:
def process_paired_rows(grid):
    """
    Processes rows with the pattern:
    #, followed by |, ending with #, where the row below is fully supportive.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 1):  # Stop at the second-to-last row
        row = grid[y]
        below_row = grid[y + 1]
        
        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a valid segment of | bounded by #
                while x < width and row[x] == '|':
                    x += 1
                
                if x < width and row[x] == '#':  # End boundary found
                    end = x
                    
                    # Verify the below row has matching boundaries
                    if below_row[start] == '#' and below_row[end] == '#' and all(below_row[i] == '#' for i in range(start + 1, end)):
                        # Replace | with ~ in the current row
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

# Example usage with the main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    
    # Apply the pattern processing function
    process_paired_rows(grid)
    
    display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Test Example
if __name__ == "__main__":
    test_grid = [
        list(".........#||||||||||||||||||||||#......................"),
        list(".........########################......................")
    ]
    print("Before Processing:")
    display_grid(test_grid)
    process_paired_rows(test_grid)
    print("After Processing:")
    display_grid(test_grid)


Before Processing:
.........#||||||||||||||||||||||#......................
.........########################......................

After Processing:
.........#~~~~~~~~~~~~~~~~~~~~~~#......................
.........########################......................



In [2]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def fill_enclosed_water(grid):
    """
    Converts flowing water ('|') to settled water ('~') in enclosed areas.
    """
    for y in range(len(grid)):
        row = grid[y]
        x = 0
        while x < len(row):
            # Find the start of a potential enclosed area
            if row[x] == '|':
                start = x
                # Check for an enclosure by moving to the right
                while x < len(row) and row[x] == '|':
                    x += 1
                end = x
                
                # Verify the enclosure
                if start > 0 and row[start - 1] == '#' and end < len(row) and row[end] == '#':
                    # Convert enclosed flowing water to settled water
                    for i in range(start, end):
                        row[i] = '~'
            else:
                x += 1

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Check if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            left_blocked, right_blocked = False, False
            
            # Spread left
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
                    queue.append((left, y))
                    break
            else:
                left_blocked = grid[y][left - 1] in ('#', '~')

            # Spread right
            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
                    queue.append((right, y))
                    break
            else:
                right_blocked = grid[y][right + 1] in ('#', '~')

            # If the row is enclosed, convert to settled water
            if left_blocked and right_blocked:
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling

    # After simulation, convert remaining enclosed water
    fill_enclosed_water(grid)

def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))

# Main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
if __name__ == "__main__":
    main('sample-input.txt')


......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
...|||||||||..
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..

Total water tiles (~ or |): 57


In [3]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def fill_enclosed_water(grid):
    """
    Converts flowing water ('|') to settled water ('~') in enclosed areas.
    """
    for y in range(len(grid)):
        row = grid[y]
        x = 0
        while x < len(row):
            # Find the start of a potential enclosed area
            if row[x] == '|':
                start = x
                # Check for an enclosure by moving to the right
                while x < len(row) and row[x] == '|':
                    x += 1
                end = x
                
                # Verify the enclosure
                if start > 0 and row[start - 1] == '#' and end < len(row) and row[end] == '#':
                    # Convert enclosed flowing water to settled water
                    for i in range(start, end):
                        row[i] = '~'
            else:
                x += 1

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Check if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            left_blocked, right_blocked = False, False
            
            # Spread left
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, mark as unblocked
                    queue.append((left, y))
                    break
            else:
                left_blocked = grid[y][left - 1] in ('#', '~')

            # Spread right
            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, mark as unblocked
                    queue.append((right, y))
                    break
            else:
                right_blocked = grid[y][right + 1] in ('#', '~')

            # If the row is enclosed, convert to settled water
            if left_blocked and right_blocked:
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling

    # After simulation, convert remaining enclosed water
    fill_enclosed_water(grid)

def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))

# Main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    # display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
if __name__ == "__main__":
    main('input.txt')


Total water tiles (~ or |): 930


In [4]:
def fill_enclosed_water(grid):
    """
    Converts flowing water ('|') to settled water ('~') in enclosed areas.
    """
    for y in range(len(grid)):
        row = grid[y]
        x = 0
        while x < len(row):
            # Find the start of a potential enclosed area
            if row[x] == '|':
                start = x
                # Check for an enclosure by moving to the right
                while x < len(row) and row[x] == '|':
                    x += 1
                end = x
                
                # Verify the enclosure
                if start > 0 and row[start - 1] == '#' and end < len(row) and row[end] == '#':
                    # Convert enclosed flowing water to settled water
                    for i in range(start, end):
                        row[i] = '~'
            else:
                x += 1

def display_grid(grid):
    """Prints the grid."""
    for row in grid:
        print(''.join(row))
    print()

# Initial grid setup
grid = [
    list("...................|..........................."),
    list("...................|..........................."),
    list(".........#.........|............#.............."),
    list(".........#.........|............#.............."),
    list(".........#...||||||||...........#.............."),
    list(".........#...|######|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|######|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#||||||||||||||||||||||#.............."),
    list(".........########################.............."),
]

# Display the initial grid
print("Initial Grid:")
display_grid(grid)

# Fill enclosed water
fill_enclosed_water(grid)

# Display the updated grid
print("Updated Grid:")
display_grid(grid)


Initial Grid:
...................|...........................
...................|...........................
.........#.........|............#..............
.........#.........|............#..............
.........#...||||||||...........#..............
.........#...|######|...........#..............
.........#...|#....#|...........#..............
.........#...|#....#|...........#..............
.........#...|#....#|...........#..............
.........#...|######|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#||||||||||||||||||||||#..............
.........########################..............

Updated Grid:
...................|...........................
...................|...........................
.........#.........|............#..............
.........#.........|............#..............
.........#...||||||||...........#..............
.........#...|######|...........#..............
.........#.

In [6]:
def transform_pattern(grid):
    """
    Transforms rows of the grid based on the given pattern:
    # falling water/sand # followed by a row of settled water.
    """
    for y in range(len(grid) - 1):  # Stop at the second last row
        current_row = grid[y]
        below_row = grid[y + 1]
        x = 0
        while x < len(current_row):
            # Look for an enclosed section of falling water/sand in the current row
            if current_row[x] == '#' and x + 1 < len(current_row):
                start = x
                x += 1
                while x < len(current_row) and current_row[x] in {'.', '|'}:
                    x += 1
                if x < len(current_row) and current_row[x] == '#':
                    end = x
                    
                    # Check if the below row is entirely settled water and enclosed
                    if below_row[start + 1:end].count('~') == (end - start - 1):
                        # Transform the enclosed section into settled water
                        for i in range(start + 1, end):
                            current_row[i] = '~'
            else:
                x += 1

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

# Initial grid setup
grid = [
    list("...................|..........................."),
    list("...................|..........................."),
    list(".........#.........|............#.............."),
    list(".........#.........|............#.............."),
    list(".........#...||||||||...........#.............."),
    list(".........#...|######|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|######|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#~~~~~~~~~~~~~~~~~~~~~~#.............."),
    list(".........########################.............."),
]

# Display the initial grid
print("Initial Grid:")
display_grid(grid)

# Apply the transformation
transform_pattern(grid)
print("After Transformation:")
display_grid(grid)


Initial Grid:
...................|...........................
...................|...........................
.........#.........|............#..............
.........#.........|............#..............
.........#...||||||||...........#..............
.........#...|######|...........#..............
.........#...|#....#|...........#..............
.........#...|#....#|...........#..............
.........#...|#....#|...........#..............
.........#...|######|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#~~~~~~~~~~~~~~~~~~~~~~#..............
.........########################..............

After Transformation:
...................|...........................
...................|...........................
.........#.........|............#..............
.........#.........|............#..............
.........#...||||||||...........#..............
.........#...|######|...........#..............
...

In [7]:
def transform_vertical_streaks(grid):
    """
    Generalizes the transformation by replacing multiple consecutive rows of falling water ('|')
    that are identical and enclosed by '#' with settled water ('~').
    """
    height = len(grid)
    width = len(grid[0])

    for y in range(height - 1):  # Stop before the last row
        # Identify the row below
        below_row = grid[y + 1]
        
        # Check for enclosed segments
        enclosed_start = None
        for x in range(width):
            if below_row[x] == '#' and enclosed_start is None:
                enclosed_start = x
            elif below_row[x] != '#' and enclosed_start is not None:
                # We've reached the end of an enclosed segment
                enclosed_end = x
                
                # Verify the vertical streak in the segment
                valid = True
                for i in range(y, -1, -1):  # Check upward rows
                    if not all(grid[i][j] == '|' for j in range(enclosed_start + 1, enclosed_end)):
                        valid = False
                        break
                if valid:
                    # Replace all falling water in the vertical streak with settled water
                    for i in range(y, -1, -1):
                        for j in range(enclosed_start + 1, enclosed_end):
                            grid[i][j] = '~'
                enclosed_start = None  # Reset for next segment

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

# Initial grid setup
grid = [
    list("...................|..........................."),
    list("...................|..........................."),
    list(".........#.........|............#.............."),
    list(".........#.........|............#.............."),
    list(".........#...||||||||...........#.............."),
    list(".........#...|######|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|#....#|...........#.............."),
    list(".........#...|######|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#~~~~~~~~~~~~~~~~~~~~~~#.............."),
    list(".........########################.............."),
]

# Display initial grid
print("Initial Grid:")
display_grid(grid)

# Apply the generalized transformation
transform_vertical_streaks(grid)
print("After Transformation:")
display_grid(grid)


Initial Grid:
...................|...........................
...................|...........................
.........#.........|............#..............
.........#.........|............#..............
.........#...||||||||...........#..............
.........#...|######|...........#..............
.........#...|#....#|...........#..............
.........#...|#....#|...........#..............
.........#...|#....#|...........#..............
.........#...|######|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#~~~~~~~~~~~~~~~~~~~~~~#..............
.........########################..............

After Transformation:
...................|...........................
...................|...........................
...

In [11]:
def transform_to_standing_water(grid):
    """
    Replaces rows containing only '|' or '|' and '.' bounded by '#' with standing water ('~'),
    provided the row below fully supports the water with '#' or '~'.
    """
    height = len(grid)
    width = len(grid[0])
    
    for y in range(height - 2, -1, -1):  # Start from the second last row and move upward
        row = grid[y]
        below_row = grid[y + 1]
        
        if '|' not in row:  # Skip rows with no falling water
            continue

        x = 0
        while x < width:
            # Look for a starting boundary
            if row[x] == '#':
                start = x
                x += 1
                
                # Check for a segment of valid characters ('|' and optionally '.')
                while x < width and row[x] in {'|', '.'}:
                    x += 1
                
                if x < width and row[x] == '#':  # End boundary found
                    end = x
                    
                    # Check if the row below supports the water
                    if all(below_row[i] in {'#', '~'} for i in range(start + 1, end)):
                        # Replace all characters between the boundaries with '~'
                        for i in range(start + 1, end):
                            row[i] = '~'
            else:
                x += 1

def display_grid(grid):
    """
    Prints the grid row by row.
    """
    for row in grid:
        print(''.join(row))
    print()

# Initial grid setup
grid = [
    list(".........#......................#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#......................#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#...|......|...........#.............."),
    list(".........#||||||||||||||||||||||#.............."),
    list(".........########################.............."),
]

# Display the initial grid
print("Initial Grid:")
display_grid(grid)

# Apply the transformation
transform_to_standing_water(grid)
print("After Transformation:")
display_grid(grid)


Initial Grid:
.........#......................#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#......................#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#||||||||||||||||||||||#..............
.........########################..............

After Transformation:
.........#......................#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#...|......|...........#..............
.........#......................#..............
.........#~~~~~~~~~~~~~~~~~~~~~~#..............
.........#~~~~~~~~~~~~~~~~~~~~~~#..............
.........#~~~~~~~~~~~~~~~~~~~~~~#..............
.........#~~~~~~~~~~~~~~~~~~~~~~#..............
.........########################..............



In [83]:
import re
from collections import deque

def parse_input(input_data):
    clay = set()
    for line in input_data.strip().split('\n'):
        if line.startswith('x='):
            x, ys = line.split(', ')
            x = int(x.split('=')[1])
            y_start, y_end = map(int, ys.split('=')[1].split('..'))
            for y in range(y_start, y_end + 1):
                clay.add((x, y))
        elif line.startswith('y='):
            y, xs = line.split(', ')
            y = int(y.split('=')[1])
            x_start, x_end = map(int, xs.split('=')[1].split('..'))
            for x in range(x_start, x_end + 1):
                clay.add((x, y))
    return clay

def create_grid(clay):
    min_x = min(x for x, y in clay) - 1
    max_x = max(x for x, y in clay) + 1
    max_y = max(y for x, y in clay)
    grid = [['.' for _ in range(max_x - min_x + 1)] for _ in range(max_y + 1)]
    for x, y in clay:
        grid[y][x - min_x] = '#'
    return grid, min_x, max_y

def display_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

def simulate_water(grid, min_x, max_y):
    spring = (500 - min_x, 0)
    grid[spring[1]][spring[0]] = '+'
    queue = deque([spring])
    
    while queue:
        x, y = queue.popleft()
        if y > max_y:
            continue

        # Flow downward
        if y + 1 <= max_y and grid[y + 1][x] == '.':
            grid[y + 1][x] = '|'
            queue.append((x, y + 1))
            continue

        # Spread left and right if downward flow is blocked
        if y + 1 <= max_y and grid[y + 1][x] in ('#', '~'):
            left, right = x, x
            while grid[y][left - 1] == '.':
                left -= 1
                grid[y][left] = '|'
                if grid[y + 1][left] == '.':  # If it can fall, add to queue
                    queue.append((left, y))
                    break

            while grid[y][right + 1] == '.':
                right += 1
                grid[y][right] = '|'
                if grid[y + 1][right] == '.':  # If it can fall, add to queue
                    queue.append((right, y))
                    break

            # Add left and right edges to queue for further processing
            if grid[y][left - 1] in ('#', '~') and grid[y][right + 1] in ('#', '~'):
                for i in range(left, right + 1):
                    grid[y][i] = '~'
                queue.append((x, y - 1))  # Recheck row above for settling

    # Post-process: Convert bounded '|' to '~'
    while True:
        changes = False
        for y in range(max_y + 1):
            row = ''.join(grid[y])
            # Replace bounded '|' sequences with '~'
            new_row = re.sub(r'#\|+\#', lambda m: '#' + '~' * (len(m.group()) - 2) + '#', row)
            if new_row != row:
                grid[y] = list(new_row)
                changes = True

        if not changes:
            break





def count_water_tiles(grid):
    # Convert the grid to a single string
    grid_str = ''.join(''.join(row) for row in grid)
    # Count matches for ~ or |
    return len(re.findall(r'[\~\|]', grid_str))

# Main function
def main(file_name):
    with open(file_name, 'r') as file:
        input_data = file.read()
    clay = parse_input(input_data)
    grid, min_x, max_y = create_grid(clay)
    simulate_water(grid, min_x, max_y)
    display_grid(grid)
    water_tiles = count_water_tiles(grid)
    print(f"Total water tiles (~ or |): {water_tiles}")

# Example usage
if __name__ == "__main__":
    main('input.txt')


......................................................................................................................................................................................................................................................................................................................+........................................
......................................................................................................................................................................................................................................................................................................................|........................................
........................................................................................................................................................................................................................................................................................................